In [40]:
# Import dependencies
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from joblib import dump

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import pandas as pd

# Don't need the following probably
import seaborn as sns 
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.corpus import stopwords
from nltk import word_tokenize

import nltk

# Data cleaning and prep

In [37]:
# Read in CSV File
raw_data=pd.read_csv('db/winequality-red.csv', dtype={'Zip': 'str'})
raw_data.head(2)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.0,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5


In [5]:
# Investigate the dataset
raw_data.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [7]:
# Shape of the data
raw_data.shape

(1599, 12)

In [8]:
# Check for NaN values
raw_data.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [16]:
raw_data.dtypes

fixed acidity           float64
volatile acidity        float64
citric acid             float64
residual sugar          float64
chlorides               float64
free sulfur dioxide     float64
total sulfur dioxide    float64
density                 float64
pH                      float64
sulphates               float64
alcohol                 float64
quality                   int64
dtype: object

In [9]:
raw_data["quality"].unique()

array([5, 6, 7, 4, 8, 3])

In [33]:
# Have to raplce lables from 3-8 to 0-5 
# NOTE: Had to replace the labels in two buckets, otherwise the data overlaps and gives an error
# TODO: Refactor the code to Map method 
first_replaced_data = raw_data.replace({'quality': {3: 0, 4: 1, 5: 2}})
final_replaced_data = first_replaced_data.replace({'quality': {6: 3, 7: 4, 8: 5}})

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,2
1,7.8,0.88,0.0,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,2


In [36]:
# Now check to see if our labels are 0-5 
final_replaced_data["quality"].unique()

array([2, 3, 4, 1, 5, 0])

# Create a train/test split

In [77]:
# Data Preprocessing
X = final_replaced_data.drop("quality", axis=1)
y = final_replaced_data["quality"]
print(X.shape, y.shape)

(1599, 11) (1599,)


In [78]:
# Split the data into tarining and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [79]:
X_train.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
1509,7.9,0.180,0.40,1.8,0.062,7.0,20.0,0.99410,3.28,0.70,11.1
706,7.0,0.780,0.08,2.0,0.093,10.0,19.0,0.99560,3.40,0.47,10.0
1071,7.5,0.770,0.20,8.1,0.098,30.0,92.0,0.99892,3.20,0.58,9.2
1292,5.9,0.395,0.13,2.4,0.056,14.0,28.0,0.99362,3.62,0.67,12.4
191,6.4,0.370,0.25,1.9,0.074,21.0,49.0,0.99740,3.57,0.62,9.8


# Pre-processing

In [69]:
# Scale the data using LabelEncoder and MinMaxScaler
# TODO: See if I should rather use Standard Scaler instead
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [82]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense

In [83]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [85]:
# Check the shape of y
y_train_categorical.shape

(1199, 6)

In [55]:
# Data normalization 
# X_scaler = StandardScaler().fit(X_train)
# X_train_scaled = X_scaler.transform(X_train)
# X_test_scaled = X_scaler.transform(X_test)

In [81]:
# print(y_train)

In [70]:
# One-hot encoding
# TODO: Learn why we should perform one hot encoding in this case specifically
# y_train_categorical = to_categorical(y_train)
# y_test_categorical = to_categorical(y_test)

# Deep Learning Model 

In [118]:
# Model Architecture 
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=11))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=6, activation='softmax'))

In [119]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [120]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 100)               1200      
_________________________________________________________________
dense_15 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_16 (Dense)             (None, 6)                 606       
Total params: 11,906
Trainable params: 11,906
Non-trainable params: 0
_________________________________________________________________


In [121]:
# Fit the model
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=500,
    shuffle=True,
    verbose=2
)

Epoch 1/500
1199/1199 - 0s - loss: 1.4196 - acc: 0.4470
Epoch 2/500
1199/1199 - 0s - loss: 1.1239 - acc: 0.5304
Epoch 3/500
1199/1199 - 0s - loss: 1.0500 - acc: 0.5521
Epoch 4/500
1199/1199 - 0s - loss: 1.0127 - acc: 0.5738
Epoch 5/500
1199/1199 - 0s - loss: 0.9994 - acc: 0.5813
Epoch 6/500
1199/1199 - 0s - loss: 0.9871 - acc: 0.5922
Epoch 7/500
1199/1199 - 0s - loss: 0.9775 - acc: 0.5963
Epoch 8/500
1199/1199 - 0s - loss: 0.9660 - acc: 0.6047
Epoch 9/500
1199/1199 - 0s - loss: 0.9633 - acc: 0.5997
Epoch 10/500
1199/1199 - 0s - loss: 0.9525 - acc: 0.6063
Epoch 11/500
1199/1199 - 0s - loss: 0.9501 - acc: 0.6088
Epoch 12/500
1199/1199 - 0s - loss: 0.9421 - acc: 0.6072
Epoch 13/500
1199/1199 - 0s - loss: 0.9405 - acc: 0.6097
Epoch 14/500
1199/1199 - 0s - loss: 0.9315 - acc: 0.6130
Epoch 15/500
1199/1199 - 0s - loss: 0.9309 - acc: 0.6130
Epoch 16/500
1199/1199 - 0s - loss: 0.9305 - acc: 0.6230
Epoch 17/500
1199/1199 - 0s - loss: 0.9194 - acc: 0.6197
Epoch 18/500
1199/1199 - 0s - loss: 0.92

1199/1199 - 0s - loss: 0.7172 - acc: 0.6922
Epoch 145/500
1199/1199 - 0s - loss: 0.7002 - acc: 0.7014
Epoch 146/500
1199/1199 - 0s - loss: 0.7072 - acc: 0.7023
Epoch 147/500
1199/1199 - 0s - loss: 0.7087 - acc: 0.6964
Epoch 148/500
1199/1199 - 0s - loss: 0.7217 - acc: 0.6964
Epoch 149/500
1199/1199 - 0s - loss: 0.7121 - acc: 0.7014
Epoch 150/500
1199/1199 - 0s - loss: 0.7069 - acc: 0.6997
Epoch 151/500
1199/1199 - 0s - loss: 0.6994 - acc: 0.7106
Epoch 152/500
1199/1199 - 0s - loss: 0.7044 - acc: 0.7073
Epoch 153/500
1199/1199 - 0s - loss: 0.7088 - acc: 0.7081
Epoch 154/500
1199/1199 - 0s - loss: 0.7022 - acc: 0.6981
Epoch 155/500
1199/1199 - 0s - loss: 0.6980 - acc: 0.7098
Epoch 156/500
1199/1199 - 0s - loss: 0.6913 - acc: 0.7081
Epoch 157/500
1199/1199 - 0s - loss: 0.6911 - acc: 0.7148
Epoch 158/500
1199/1199 - 0s - loss: 0.6993 - acc: 0.7014
Epoch 159/500
1199/1199 - 0s - loss: 0.7046 - acc: 0.7073
Epoch 160/500
1199/1199 - 0s - loss: 0.6950 - acc: 0.7131
Epoch 161/500
1199/1199 - 0s

Epoch 286/500
1199/1199 - 0s - loss: 0.5599 - acc: 0.7640
Epoch 287/500
1199/1199 - 0s - loss: 0.5577 - acc: 0.7623
Epoch 288/500
1199/1199 - 0s - loss: 0.5607 - acc: 0.7631
Epoch 289/500
1199/1199 - 0s - loss: 0.5528 - acc: 0.7756
Epoch 290/500
1199/1199 - 0s - loss: 0.5561 - acc: 0.7640
Epoch 291/500
1199/1199 - 0s - loss: 0.5642 - acc: 0.7665
Epoch 292/500
1199/1199 - 0s - loss: 0.5485 - acc: 0.7706
Epoch 293/500
1199/1199 - 0s - loss: 0.5590 - acc: 0.7515
Epoch 294/500
1199/1199 - 0s - loss: 0.5438 - acc: 0.7715
Epoch 295/500
1199/1199 - 0s - loss: 0.5435 - acc: 0.7740
Epoch 296/500
1199/1199 - 0s - loss: 0.5414 - acc: 0.7731
Epoch 297/500
1199/1199 - 0s - loss: 0.5443 - acc: 0.7781
Epoch 298/500
1199/1199 - 0s - loss: 0.5507 - acc: 0.7698
Epoch 299/500
1199/1199 - 0s - loss: 0.5594 - acc: 0.7590
Epoch 300/500
1199/1199 - 0s - loss: 0.5420 - acc: 0.7840
Epoch 301/500
1199/1199 - 0s - loss: 0.5386 - acc: 0.7832
Epoch 302/500
1199/1199 - 0s - loss: 0.5382 - acc: 0.7781
Epoch 303/500


Epoch 428/500
1199/1199 - 0s - loss: 0.4185 - acc: 0.8390
Epoch 429/500
1199/1199 - 0s - loss: 0.4178 - acc: 0.8357
Epoch 430/500
1199/1199 - 0s - loss: 0.4192 - acc: 0.8382
Epoch 431/500
1199/1199 - 0s - loss: 0.4175 - acc: 0.8390
Epoch 432/500
1199/1199 - 0s - loss: 0.4164 - acc: 0.8332
Epoch 433/500
1199/1199 - 0s - loss: 0.4263 - acc: 0.8299
Epoch 434/500
1199/1199 - 0s - loss: 0.4100 - acc: 0.8357
Epoch 435/500
1199/1199 - 0s - loss: 0.4194 - acc: 0.8365
Epoch 436/500
1199/1199 - 0s - loss: 0.4277 - acc: 0.8315
Epoch 437/500
1199/1199 - 0s - loss: 0.4016 - acc: 0.8482
Epoch 438/500
1199/1199 - 0s - loss: 0.4034 - acc: 0.8424
Epoch 439/500
1199/1199 - 0s - loss: 0.4094 - acc: 0.8515
Epoch 440/500
1199/1199 - 0s - loss: 0.4029 - acc: 0.8374
Epoch 441/500
1199/1199 - 0s - loss: 0.4101 - acc: 0.8515
Epoch 442/500
1199/1199 - 0s - loss: 0.4078 - acc: 0.8449
Epoch 443/500
1199/1199 - 0s - loss: 0.3977 - acc: 0.8507
Epoch 444/500
1199/1199 - 0s - loss: 0.4016 - acc: 0.8457
Epoch 445/500


In [122]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

400/400 - 0s - loss: 1.5385 - acc: 0.6050
Normal Neural Network - Loss: 1.5384754180908202, Accuracy: 0.6050000190734863


In [123]:
encoded_predictions = model.predict_classes(X_test_scaled[:10])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [124]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:10])}")

Predicted classes: [3 2 4 2 2 4 2 2 2 2]
Actual Labels: [3, 2, 5, 1, 2, 4, 2, 2, 2, 4]


# Random Forest 

In [128]:
rf = RandomForestClassifier(n_estimators=500)
rf = rf.fit(X_train_scaled, y_train)
rf_score = rf.score(X_test_scaled, y_test)
print(rf_score)

0.6975


In [ ]:
# NOTE to convert into percentages 
##############################################################################
# Obtain Accuracy Scores for the test set
##############################################################################
print('Decision Tree:', accuracy_score(y_test, tree_pred)*100,'%')
print('Random Forest:', accuracy_score(y_test, rf_pred)*100,'%')
print('KNeighbors:',accuracy_score(y_test, KN_pred)*100,'%')
print('GaussianNB:',accuracy_score(y_test, Gaussian_pred)*100,'%')
print('SVC:',accuracy_score(y_test, svc_pred)*100,'%')
print('XGB:',accuracy_score(y_test, xgb_pred)*100,'%')